## MAC0317/MAC5920
## Introdução ao Processamento de Sinais Digitais
### Seção 1.6: Amostragem e aliasing

In [ ]:
import math as m
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from IPython.display import HTML

### Dependência do sinal em relação à taxa de amostragem
### Figura 1.8: Exemplo simples de sinal 1D amostrado a 20Hz ($\Delta_t=0.05$)

$$f(t) = \sin(\omega t)$$

In [ ]:
t = np.linspace(0,1,21)
x = np.sin(44*m.pi*t)
plt.figure(figsize=(15,5))
plt.axhline(y=0.0, color='gray', linestyle='--', ms=1)
plt.plot(t,x,"*")
plt.show()

### Figura 1.9: A mesma função anterior, amostrada a 200Hz ($\Delta_t=0.005$)

In [ ]:
t = np.linspace(0,1,21)
x = np.sin(44*m.pi*t)
t2 = np.linspace(0,1,201)
x2 = np.sin(44*m.pi*t2)
plt.figure(figsize=(15,5))
plt.axhline(y=0.0, color='gray', linestyle='--', ms=1)
plt.plot(t2,x2, label='amostrado a 200Hz', color='orange')
plt.plot(t,x,"*", label='amostrado a 20Hz')
plt.xlim([0, 1])
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

## Explicação: o fenômeno do rebatimento

**O problema:** existem infinitos valores de $f$ e $g$ tais que

$$\sin\left(2\pi f\frac{n}{R}\right) = \sin\left(2\pi g\frac{n}{R}\right), \forall n\in\mathbb{Z}.$$

Para essa condição valer, basta que os argumentos das senoides acima correspondam sempre a ângulos equivalentes. Isso ocorre, por exemplo, quando

$$2\pi f\frac{n}{R} = 2\pi g\frac{n}{R} + k*2\pi*n$$

para algum $k\in\mathbb{Z}$ e para qualquer $n\in\mathbb{Z}$. A condição acima equivale a

$$f-g = kR.$$

A mesma condição vale para que dois cossenos de frequências $f$ e $g$ produzam a mesma lista de amostras, e analogamente para duas exponenciais complexas de frequências f e g:

$$\begin{array}{c}
e^{i2\pi f\frac{n}{R}} = e^{i2\pi g\frac{n}{R}}, \forall n\in\mathbb{Z}\\
\Updownarrow\\
e^{i2\pi (f-g)\frac{n}{R}}=1, \forall n\in\mathbb{Z}\\
\Updownarrow\\
e^{i2\pi (f-g)\frac{n}{R}}=e^{i2\pi n}, \forall n\in\mathbb{Z}\\
\Updownarrow\\
\forall n\in\mathbb{Z}\quad\exists k\in\mathbb{Z}: 2\pi (f-g)\frac{n}{R} = k2\pi n\\
\Updownarrow\\
\exists k\in\mathbb{Z}: f-g = kR
\end{array}
$$

Assim, ao utilizarmos uma taxa de amostragem R, todas as senoides e/ou exponenciais com frequências

$$g=\ldots,f-3R,f-2R,f-R,f,f+R,f+2R,f+3R\ldots$$

gerarão a mesma lista de amostras que aquela com frequência $f$.

### Aliasing traz duas implicações importantes:

> - implicação física/perceptual: ao se amostrar um sinal analógico com taxa de amostragem $R$, perde-se a capacidade de discernir frequências separadas por múltiplos de $R$, e em particular os valores que não estão na faixa $-\frac{R}{2}<f\le \frac{R}{2}$ serão representados incorretamente (rebatidos), correspondendo a alguma outra frequência (aparente) que pertence à faixa representável.

> - implicação matemática: para se analisar um sinal a $R$ Hz, basta considerar frequências na faixa $-\frac{R}{2}<f\le \frac{R}{2}$.

Essas implicações ilustram a condição de Shannon-Nyquist, que afirma que para representar corretamente componentes senoidais de até $F$ Hz é necessário utilizar uma taxa de amostragem $R\ge 2F$ Hz, e também explicam a necessidade de utilização de filtros passa-baixas na conversão analógico-digital.



### Figura 1.10: Exemplo de aliasing espacial. A mesma função 

$$f(x,y)=\sin(2\pi(50x+70y))$$

### é amostrada em grids de 60x60, 100x100, 300x300 e 1000x1000

In [ ]:
N=[60, 100, 300, 1000]
g, ax = plt.subplots(1,4,figsize=(18,5))
for j in range(4):
    R = N[j]; x = np.arange(0,1,1/R); y = np.arange(0,1,1/R)
    x, y = np.meshgrid(x,y)
    # só de brincadeira, construímos o seno usando a parte imaginária de uma exponencial complexa
    f = m.e ** (2 * m.pi * 1j * 70 * y) * m.e ** (2 * m.pi * 1j * 50 * x)
    ax[j].imshow(f.imag, origin='lower', cmap='gray', vmin=-1, vmax=1)
    ax[j].axis('off')
    ax[j].set_title("amostragem com {0}x{1} pontos".format(N[j],N[j]))
plt.show()

### Alternativa: mostra a variação do tamanho do grid de 10 em 10

In [ ]:
%%capture
plt.ion()
fig = plt.figure(figsize=(10,10))
plt.gray()
density = 500
x = np.arange(0,1,1/density)
y = np.arange(0,1,1/density)
x, y = np.meshgrid(x,y)
ims = []
for N in np.arange(50,500,10):
    f = m.e**(2*m.pi*1j*70*np.round(y*N)/N)*m.e**(2*m.pi*1j*50*np.round(x*N)/N)
    im = plt.imshow(f.imag, origin='lower', cmap='gray', vmin=-1, vmax=1)
    plt.axis('off')
    ims.append([im])
    
imf = anim.ArtistAnimation(fig, ims, interval=250, blit=True)

In [ ]:
HTML(imf.to_jshtml())